# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# load in the data
azdias = pd.read_csv('./data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('./data/Udacity_CUSTOMERS_052018.csv', sep=';')

/Users/boonchuay/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
# specify path and filename for metadata
MET_DIR = Path('./metadata')
MET_FILENAMES = ['age-data.json', 'interval-data.json', 'nominal-data.json', 'ordinal-data.json']

# load json file to metadata dict
metadata = {}

for file_ in MET_FILENAMES:
    with open(MET_DIR / file_, 'r') as fp:
        print('load --> ', file_.replace('.json', ''))
        metadata[file_.replace('.json', '')] = json.load(fp)

load -->  age-data
load -->  interval-data
load -->  nominal-data
load -->  ordinal-data


## Check consistency in metadata file feature description and existing feature in raw data

In [31]:
def check_containment(metadata, df):
    '''check that all column in metadata is existed
    '''
    not_contains = []

    for data_type, metas in metadata.items():
        print(f'===={data_type}====')

        for meta in metas:
            feature = meta['feature']

            if feature not in df.columns:
                print(feature)
                not_contains.append(feature)

    return not_contains

def check_containment_reverse(metada, df):
    '''Check that all column in df are exist in metada
    '''

    meta_features = []

    for data_type, metas in metadata.items():

        for meta in metas:
            feature = meta['feature']
            meta_features.append(feature)

    print(len(meta_features))
    not_contains = []
    for feature in df.columns:
        if feature not in meta_features:
            print(feature)
            not_contains.append(feature)

    return not_contains


In [32]:
data_not_contains = check_containment(metadata, azdias)

====age-data====
====interval-data====
====nominal-data====
SOHO_FLAG
HAUSHALTSSTRUKTUR
BIP_FLAG
CAMEO_DEUINTL_2015
====ordinal-data====
WACHSTUMSGEBIET_NB
GEOSCORE_KLS7


In [33]:
meta_not_contains = check_containment_reverse(metadata, azdias)

205
LNR
AKT_DAT_KL
ALTER_KIND1
ALTER_KIND2
ALTER_KIND3
ALTER_KIND4
ALTERSKATEGORIE_FEIN
ANZ_KINDER
ANZ_STATISTISCHE_HAUSHALTE
ARBEIT
CAMEO_INTL_2015
CJT_KATALOGNUTZER
CJT_TYP_1
CJT_TYP_2
CJT_TYP_3
CJT_TYP_4
CJT_TYP_5
CJT_TYP_6
D19_BUCH_CD
D19_KONSUMTYP_MAX
D19_LETZTER_KAUF_BRANCHE
D19_SOZIALES
D19_TELKO_ONLINE_QUOTE_12
D19_VERSI_DATUM
D19_VERSI_OFFLINE_DATUM
D19_VERSI_ONLINE_DATUM
D19_VERSI_ONLINE_QUOTE_12
DSL_FLAG
EINGEFUEGT_AM
EINGEZOGENAM_HH_JAHR
EXTSEL992
FIRMENDICHTE
GEMEINDETYP
HH_DELTA_FLAG
KBA13_ALTERHALTER_30
KBA13_ALTERHALTER_45
KBA13_ALTERHALTER_60
KBA13_ALTERHALTER_61
KBA13_ANTG1
KBA13_ANTG2
KBA13_ANTG3
KBA13_ANTG4
KBA13_AUDI
KBA13_AUTOQUOTE
KBA13_BAUMAX
KBA13_BJ_1999
KBA13_BJ_2000
KBA13_BJ_2004
KBA13_BJ_2006
KBA13_BJ_2008
KBA13_BJ_2009
KBA13_BMW
KBA13_CCM_0_1400
KBA13_CCM_1000
KBA13_CCM_1200
KBA13_CCM_1400
KBA13_CCM_1401_2500
KBA13_CCM_1500
KBA13_CCM_1600
KBA13_CCM_1800
KBA13_CCM_2000
KBA13_CCM_2500
KBA13_CCM_2501
KBA13_CCM_3000
KBA13_CCM_3001
KBA13_FAB_ASIEN
KBA13_FAB_SON

From the above information, we can see that there are many features in metadata file (which extract from DIAS Attributes file) that are not exist in azdias's columns.  

Many features in azdias's columns are, also, not exist in metadata file. Without a description I can not know the meaning of the features, so I consider drop all the features that are not exist in metadata file.

In [69]:
def get_all_exist_feature(metadata, data_not_contains):
    '''Get all feature names from metadata file
    '''
    
    all_feature = []

    for features in [*map(lambda key: metadata[key], metadata)]:
        for feature in features:
            feature_name = feature['feature']
            all_feature.append(feature_name)

    all_feature_contains = list(set(all_feature) - set(data_not_contains))
    return all_feature_contains

In [60]:
all_feature_contains = get_all_exist_feature(metadata, data_not_contains)

In [63]:
print(all_feature_contains)

['KBA05_KW3', 'KBA05_ZUL1', 'KBA05_HERST4', 'D19_HANDWERK', 'D19_GESAMT_OFFLINE_DATUM', 'SEMIO_PFLICHT', 'D19_GARTEN', 'D19_BANKEN_ONLINE_DATUM', 'KBA05_SEG7', 'ONLINE_AFFINITAET', 'D19_KOSMETIK', 'KBA05_ANTG2', 'CAMEO_DEUG_2015', 'KBA05_KRSKLEIN', 'KBA05_KRSHERST2', 'PLZ8_GBZ', 'KBA05_ALTER3', 'D19_VERSI_ANZ_12', 'KBA05_KRSHERST1', 'FINANZ_UNAUFFAELLIGER', 'LP_STATUS_FEIN', 'D19_WEIN_FEINKOST', 'D19_TELKO_ANZ_12', 'KBA05_VORB1', 'KBA05_MAXBJ', 'KBA05_MOD8', 'D19_TELKO_MOBILE', 'D19_BANKEN_DIREKT', 'KBA05_SEG5', 'D19_SCHUHE', 'HEALTH_TYP', 'KBA05_KRSZUL', 'D19_GESAMT_ANZ_24', 'KBA05_ANTG3', 'NATIONALITAET_KZ', 'CJT_GESAMTTYP', 'ANZ_HAUSHALTE_AKTIV', 'KBA05_CCM3', 'SEMIO_KRIT', 'KBA05_BAUMAX', 'D19_BANKEN_REST', 'KKK', 'KBA05_ANTG4', 'PLZ8_ANTG1', 'VERS_TYP', 'D19_GESAMT_ONLINE_QUOTE_12', 'INNENSTADT', 'KBA05_SEG1', 'KONSUMNAEHE', 'LP_FAMILIE_FEIN', 'CAMEO_DEU_2015', 'D19_BANKEN_LOKAL', 'PLZ8_HHZ', 'FINANZ_HAUSBAUER', 'KBA05_MOD3', 'KBA05_MAXHERST', 'OST_WEST_KZ', 'D19_TELKO_ANZ_24', 'S

## Check completeness of the data

In [74]:
# create dict ['feature_name' -> unknown_value] to encode unknown value in dataframe as null
def get_feature2unk(metadata):

    feature2unk = {}

    for _, metas in metadata.items():
        for meta in metas:
            feature2unk[meta['feature']] = meta['unknown_value']

    return feature2unk

In [75]:
feature2unk = get_feature2unk(metadata)

In [82]:
# check completeness of the data
for feature in all_feature_contains:
    col = azdias[feature].copy()

    if feature2unk[feature] is not None:
        col.loc[col.isin(feature2unk[feature])] = np.nan

    print(f'{feature}: {col.notnull().sum()/len(azdias)*100:.2f}%')

KBA05_KW3: 83.38%
KBA05_ZUL1: 83.38%
KBA05_HERST4: 83.38%
D19_HANDWERK: 13.78%
D19_GESAMT_OFFLINE_DATUM: 37.33%
SEMIO_PFLICHT: 100.00%
D19_GARTEN: 4.44%
D19_BANKEN_ONLINE_DATUM: 18.43%
KBA05_SEG7: 83.38%
ONLINE_AFFINITAET: 92.08%
D19_KOSMETIK: 16.31%
KBA05_ANTG2: 85.04%
CAMEO_DEUG_2015: 88.89%
KBA05_KRSKLEIN: 83.38%
KBA05_KRSHERST2: 83.38%
PLZ8_GBZ: 86.93%
KBA05_ALTER3: 83.38%
D19_VERSI_ANZ_12: 7.85%
KBA05_KRSHERST1: 83.38%
FINANZ_UNAUFFAELLIGER: 100.00%
LP_STATUS_FEIN: 99.46%
D19_WEIN_FEINKOST: 6.18%
D19_TELKO_ANZ_12: 3.73%
KBA05_VORB1: 83.38%
KBA05_MAXBJ: 83.38%
KBA05_MOD8: 83.38%
D19_TELKO_MOBILE: 18.45%
D19_BANKEN_DIREKT: 18.22%
KBA05_SEG5: 83.38%
D19_SCHUHE: 13.26%
HEALTH_TYP: 87.52%
KBA05_KRSZUL: 83.38%
D19_GESAMT_ANZ_24: 43.30%
KBA05_ANTG3: 85.04%
NATIONALITAET_KZ: 87.85%
CJT_GESAMTTYP: 99.46%
ANZ_HAUSHALTE_AKTIV: 89.55%
KBA05_CCM3: 83.38%
SEMIO_KRIT: 100.00%
KBA05_BAUMAX: 46.53%
D19_BANKEN_REST: 7.79%
KKK: 82.26%
KBA05_ANTG4: 85.04%
PLZ8_ANTG1: 86.93%
VERS_TYP: 87.52%
D19_GESAM

We can see that, after encoding unkown value (describe in given excel file), there are some features that contains too much unknown value such as. 

* D19_RATGEBER --> missing 82.69%
* D19_LOTTO --> missing 83.92%  

I will consider removing this features in data cleaning phase.

## Observe data distribution 
to select only the feature that can differentiate the normal population with potential customers

In [96]:
name = 'KKK'

t1 = azdias[name].value_counts().to_numpy()
t2 = customers[name].value_counts().to_numpy()
sum(t1*t2)/(np.linalg.norm(t1, ord=2)*np.linalg.norm(t2, ord=2))

0.9843663384323514

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')